### Data Cleaning 

In [40]:
import pandas as pd
df_matches = pd.read_csv("matches.csv")
df_matches.head(3)

,id,season,city,date,team1,team2,toss_winner,toss_decision,result,dl_applied,winner,win_by_runs,win_by_wickets,player_of_match,venue,umpire1,umpire2,umpire3
0,1,2017,Hyderabad,2017-04-05,Sunrisers Hyderabad,Royal Challengers Bangalore,Royal Challengers Bangalore,field,normal,0,Sunrisers Hyderabad,35,0,Yuvraj Singh,"Rajiv Gandhi International Stadium, Uppal",AY Dandekar,NJ Llong,NaN
1,2,2017,Pune,2017-04-06,Mumbai Indians,Rising Pune Supergiant,Rising Pune Supergiant,field,normal,0,Rising Pune Supergiant,0,7,SPD Smith,Maharashtra Cricket Association Stadium,A Nand Kishore,S Ravi,NaN
2,3,2017,Rajkot,2017-04-07,Gujarat Lions,Kolkata Knight Riders,Kolkata Knight Riders,field,normal,0,Kolkata Knight Riders,0,10,CA Lynn,Saurashtra Cricket Association Stadium,Nitin Menon,CK Nandan,NaN


In [41]:
df_deliveries = pd.read_csv("deliveries.csv")
df_deliveries.head(3)

,match_id,inning,batting_team,bowling_team,over,ball,batsman,non_striker,bowler,is_super_over,wide_runs,bye_runs,legbye_runs,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs,player_dismissed,dismissal_kind,fielder
0,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,1,DA Warner,S Dhawan,TS Mills,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN
1,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,2,DA Warner,S Dhawan,TS Mills,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN
2,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,3,DA Warner,S Dhawan,TS Mills,0,0,0,0,0,0,4,0,4,NaN,NaN,NaN


In [42]:
print(df_matches.shape)
print(df_deliveries.shape)

(756, 18)
(179078, 21)


In [43]:
#plotting every team wins not only RCB and KKR. 
import matplotlib.pyplot as plt
import plotly.graph_objs as go
df_winner_matches = df_matches[['id','winner']].groupby('winner').size().reset_index()
df_winner_matches.columns = ['winner','total_wins']
df_winner_matches = df_winner_matches.sort_values('total_wins',ascending=False)

data = go.Bar(x = df_winner_matches['winner'],y = df_winner_matches['total_wins'])
layout = go.Layout(title = 'Total wins by every team in IPL',xaxis = dict(title = 'Team'),yaxis = dict(title = 'No of wins'))
fig = go.Figure(data = data,layout = layout)
fig.show()

In [44]:
print(df_matches.columns)
print(df_deliveries.columns)

Index(['id', 'season', 'city', 'date', 'team1', 'team2', 'toss_winner',
       'toss_decision', 'result', 'dl_applied', 'winner', 'win_by_runs',
       'win_by_wickets', 'player_of_match', 'venue', 'umpire1', 'umpire2',
       'umpire3'],
      dtype='object')
Index(['match_id', 'inning', 'batting_team', 'bowling_team', 'over', 'ball',
       'batsman', 'non_striker', 'bowler', 'is_super_over', 'wide_runs',
       'bye_runs', 'legbye_runs', 'noball_runs', 'penalty_runs',
       'batsman_runs', 'extra_runs', 'total_runs', 'player_dismissed',
       'dismissal_kind', 'fielder'],
      dtype='object')


In matches data frame columns like umpire1,umpire2,umpire3 are not useful because these days they are using technology. So we cant decide which umpire is more useful for predicting anything related to match.

Date is also not useful because we already have season column which is also almost equal to date.

City and venue also not important features for this year IPL. Because IPL2020 is happening in UAE, only 2014 ipl was happened in UAE that to half season. But i am removing only venue it almost equal to city feature.

'Result' feature is almost equal to 'winner' feature. So i am dropping result column also.

In [45]:
df_matches = df_matches.drop(['venue','date','result','umpire1','umpire2','umpire3'], axis = 1) 
df_matches.head(2)

,id,season,city,team1,team2,toss_winner,toss_decision,dl_applied,winner,win_by_runs,win_by_wickets,player_of_match
0,1,2017,Hyderabad,Sunrisers Hyderabad,Royal Challengers Bangalore,Royal Challengers Bangalore,field,0,Sunrisers Hyderabad,35,0,Yuvraj Singh
1,2,2017,Pune,Mumbai Indians,Rising Pune Supergiant,Rising Pune Supergiant,field,0,Rising Pune Supergiant,0,7,SPD Smith


In [46]:
print(df_matches["team2"].value_counts())

Royal Challengers Bangalore    95
Kolkata Knight Riders          95
Delhi Daredevils               89
Mumbai Indians                 86
Kings XI Punjab                85
Rajasthan Royals               80
Chennai Super Kings            75
Sunrisers Hyderabad            45
Deccan Chargers                32
Pune Warriors                  26
Gujarat Lions                  16
Delhi Capitals                 10
Rising Pune Supergiant          8
Kochi Tuskers Kerala            7
Rising Pune Supergiants         7
Name: team2, dtype: int64


In [47]:
#In 2019 delhi daredevils name changed to delhi capitals. So i am replacing old name with new name for all the rows.
df_matches = df_matches.replace(to_replace ="Delhi Daredevils", value = "Delhi Capitals") 

df_deliveries = df_deliveries.replace(to_replace = "Delhi Daredevils", value = "Delhi Capitals")

In [48]:
# We are predicting results of the match between DC and RR. So i am taking only rows which has DC or RR from team1 and team2 columns in matches dataset.
df_matches = df_matches[(df_matches['team1'] == 'Delhi Capitals') | (df_matches['team1'] == 'Rajasthan Royals') | 
                        (df_matches['team2'] == 'Delhi Capitals') | (df_matches['team2'] == 'Rajasthan Royals')]


df_matches.head(2)

,id,season,city,team1,team2,toss_winner,toss_decision,dl_applied,winner,win_by_runs,win_by_wickets,player_of_match
4,5,2017,Bangalore,Royal Challengers Bangalore,Delhi Capitals,Royal Challengers Bangalore,bat,0,Royal Challengers Bangalore,15,0,KM Jadhav
8,9,2017,Pune,Delhi Capitals,Rising Pune Supergiant,Rising Pune Supergiant,field,0,Delhi Capitals,97,0,SV Samson


In [49]:
#checking null values in matches dataset
print(df_matches.isnull().sum())

id                 0
season             0
city               3
team1              0
team2              0
toss_winner        0
toss_decision      0
dl_applied         0
winner             4
win_by_runs        0
win_by_wickets     0
player_of_match    4
dtype: int64


In [50]:
#printing null rows in matches dataset
df1_null = df_matches[df_matches.isna().any(axis=1)]
df1_null.head()

,id,season,city,team1,team2,toss_winner,toss_decision,dl_applied,winner,win_by_runs,win_by_wickets,player_of_match
300,301,2011,Delhi,Delhi Capitals,Pune Warriors,Delhi Capitals,bat,0,NaN,0,0,NaN
462,463,2014,NaN,Kolkata Knight Riders,Delhi Capitals,Kolkata Knight Riders,bat,0,Delhi Capitals,0,4,JP Duminy
466,467,2014,NaN,Chennai Super Kings,Rajasthan Royals,Rajasthan Royals,field,0,Chennai Super Kings,7,0,RA Jadeja
468,469,2014,NaN,Sunrisers Hyderabad,Delhi Capitals,Sunrisers Hyderabad,bat,0,Sunrisers Hyderabad,4,0,AJ Finch
545,546,2015,Bangalore,Royal Challengers Bangalore,Rajasthan Royals,Rajasthan Royals,field,0,NaN,0,0,NaN


In [51]:
#After seeing above table we can clearly understood that 4 matches are not played beacuse of rain, badlight or pitch condition. 
#So i am replacing those null values in winner column with no result and palyer of the match column null values with not declared.
df_matches["winner"].fillna("No result", inplace = True)
df_matches['player_of_match'].fillna("Not declared", inplace = True)
#city column also has 3 null values. I checked those matches data in espn website and they played in Dubai.
df_matches["city"].fillna("Dubai", inplace = True)
print(df_matches.shape)

(304, 12)


In [52]:
#checking null values in deliveries dataset
print(df_deliveries.isnull().sum())

match_id                 0
inning                   0
batting_team             0
bowling_team             0
over                     0
ball                     0
batsman                  0
non_striker              0
bowler                   0
is_super_over            0
wide_runs                0
bye_runs                 0
legbye_runs              0
noball_runs              0
penalty_runs             0
batsman_runs             0
extra_runs               0
total_runs               0
player_dismissed    170244
dismissal_kind      170244
fielder             172630
dtype: int64


Player_dismissed, dismissal_kind and fielder are having null values. But if you consider today questions only player_dismissed is useful for calculating average of batsman. So i am removing remaining two features.

Filling player_dismissed null values with no wicket

In [53]:
#dropping columns
df_deliveries = df_deliveries.drop(['dismissal_kind','fielder'], axis = 1) 
#filling null values
df_deliveries["player_dismissed"].fillna("No Wicket", inplace = True)
df_deliveries.head(2)

,match_id,inning,batting_team,bowling_team,over,ball,batsman,non_striker,bowler,is_super_over,wide_runs,bye_runs,legbye_runs,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs,player_dismissed
0,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,1,DA Warner,S Dhawan,TS Mills,0,0,0,0,0,0,0,0,0,No Wicket
1,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,2,DA Warner,S Dhawan,TS Mills,0,0,0,0,0,0,0,0,0,No Wicket


### Exploratory Data Analysis(EDA)

### Q1) Who will win the match

In [98]:
#plotting DC and RR wins against all teams
import plotly.graph_objs as go
df_winner = df_matches[['id','winner']].groupby('winner').size().reset_index()
df_winner.columns = ['winner','total_wins']
df_winner = df_winner.sort_values('total_wins',ascending=False)[:2]

data = go.Bar(x = df_winner['winner'],y = df_winner['total_wins'])
layout = go.Layout(title = 'Total wins by DC and RR in IPL',xaxis = dict(title = 'Team'),yaxis = dict(title = 'No of wins'))
fig = go.Figure(data = data,layout = layout)
fig.show()

Number of wins by Delhi and Rajasthan are almost equal in IPL. Delhi won 2more matches than RR but remember Rajasthan royals not played 2 ipl seasons. 

In [55]:
# Lets check winning ratio between DC VS RR
df_DC_RR = df_matches[((df_matches['team1'] == 'Delhi Capitals') | (df_matches['team1'] == 'Rajasthan Royals')) & 
                        ((df_matches['team2'] == 'Delhi Capitals') | (df_matches['team2'] == 'Rajasthan Royals'))] 

#plotting DC and RR wins against each other
import plotly.graph_objs as go
df_winner = df_DC_RR[['id','winner']].groupby('winner').size().reset_index()
df_winner.columns = ['winner','total_wins']
df_winner = df_winner.sort_values('total_wins',ascending=False)[:2]

data = go.Bar(x = df_winner['winner'],y = df_winner['total_wins'])
layout = go.Layout(title = 'Number of wins by DC and RR against each other',xaxis = dict(title = 'Team'),yaxis = dict(title = 'No of wins'))
fig = go.Figure(data = data,layout = layout)
fig.show() 

In head-head Rajasthan won 3 more matches than Delhi. 

But we cant predict by using above two observations. Because delhi won more matches in ipl but RR won more matches against DC in IPL. One more thing is players will change every 2 ipl seasons. So lets look at individual player records.

In [56]:
#Lets look at how many player of the match awards won by current DC and RR players.
#Note: I am taking top 8 palyers from both teams
df_man_of_match_DC = df_matches[(df_matches['player_of_match'] == 'P Shaw') | (df_matches['player_of_match'] == 'S Dhawan') |
                                (df_matches['player_of_match'] == 'SS Iyer') | (df_matches['player_of_match'] == 'RR Pant') |
                                (df_matches['player_of_match'] == 'R Ashwin') | (df_matches['player_of_match'] == 'K Rabada') |
                                (df_matches['player_of_match'] == 'AR Patel') | (df_matches['player_of_match'] == 'H Patel')]

print("Total number of man of matches won by Delhi players", df_man_of_match_DC.shape[0])

df_man_of_match_RR = df_matches[(df_matches['player_of_match'] == 'JC Buttler') | (df_matches['player_of_match'] == 'SPD Smith') |
                                (df_matches['player_of_match'] == 'SV Samson') | (df_matches['player_of_match'] == 'BA Stokes') |
                                (df_matches['player_of_match'] == 'J Archer') | (df_matches['player_of_match'] == 'S Gopal') |
                                (df_matches['player_of_match'] == 'R Tewatia') | (df_matches['player_of_match'] == 'RV Uthappa') ]
print("Total number of man of matches won by Delhi players", df_man_of_match_RR.shape[0])

Total number of man of matches won by Delhi players 18
Total number of man of matches won by Delhi players 14


It's clearly indicating that Rajasthan players won more man of match awards than Delhi players. 

So we can say that Rajasthan has more match winners than Delhi and also in Rajasthan won more matches against Delhi.

I am predicting that Rajasthan Royals will win today match.

### Q2) How many runs will delhi score in the powerplay

In [57]:
#Actually top 3 batsman(i.e openers and onedown batsman) will face more deliveries in powerplay.
#So i am picking top3 batsman from delhi capitals team(i.e Shaw, Dhawan, Shreyas).
df_top3_batsman_DC = df_deliveries[(df_deliveries['batsman'] == 'P Shaw') | (df_deliveries['batsman'] == 'S Dhawan') |
                                   (df_deliveries['batsman'] == 'SS Iyer')]
df_top3_batsman_DC.head()

#Now we are calculating runs scored in powerplay i.e first 6 overs by above 3 batsman. 
#So for that we can filter out 1 to 6 overs from 'over' feature/column.
df_top3_powerplay_DC = df_top3_batsman_DC[(df_top3_batsman_DC['over'] == 1) | (df_top3_batsman_DC['over'] == 2) |
                                          (df_top3_batsman_DC['over'] == 3) | (df_top3_batsman_DC['over'] == 4) |
                                          (df_top3_batsman_DC['over'] == 5) | (df_top3_batsman_DC['over'] == 6)]


In [60]:
#Now we can calcualte the batting average of DC top 3 batsman in powerplay

df_powerpaly_avg_DC = df_top3_powerplay_DC["batsman_runs"].sum() /len(df_top3_powerplay_DC[(df_top3_powerplay_DC['player_dismissed'] == 'P Shaw') |
                                                                      (df_top3_powerplay_DC['player_dismissed'] == 'S Dhawan')|
                                                                      (df_top3_powerplay_DC['player_dismissed'] == 'SS Iyer')])

print("Average runs scored by Delhi top 3 batsman in powerplay",df_powerpaly_avg_DC)

Average runs scored by Delhi top 3 batsman in powerplay 34.81443298969072


According to above observation, I am predicting Delhi Capitals will score 31-40 runs in powerplay.

### Q3) How many runs will Steve Smith score in the match

In [65]:
#Calculating Steve smith average against present Delhi bowlers.
#Actually i am taking only 4 bowlers becuase 5th bowler is Anrich Norte and he is playing his 1st ipl season.
df_Smith = df_deliveries[(((df_deliveries['batsman'] == 'SPD Smith')) & ((df_deliveries['bowler'] == 'R Ashwin') |
                                                                        (df_deliveries['bowler'] == 'K Rabada') | 
                                                                        (df_deliveries['bowler'] == 'AR Patel') |
                                                                        (df_deliveries['bowler'] == 'H Patel')))] 

print("Steve Smith runs against present DC bowlers : " , df_Smith['batsman_runs'].sum())
print("In Number of matches Steve Smith got batting aginst present DC bowlers : ",df_Smith['match_id'].nunique())
print("In Number of matches Steve Smith out against present DC bowlers : ",len(df_Smith[df_Smith['player_dismissed'] == 'SPD Smith'])) 

Steve Smith runs against present DC bowlers :  48
In Number of matches Steve Smith got batting aginst present DC bowlers :  10
In Number of matches Steve Smith out against present DC bowlers :  2


In [75]:
#By looking at above observations Steve Smith is remained not out on 8 matches out of 10 played against DC bowlers
#Now i am claculating Smith batting average against DC bowlers
Smith_average = df_Smith['batsman_runs'].sum()/len(df_Smith[df_Smith['player_dismissed'] == 'SPD Smith'])
print("Steve Smith average against DC bowlers : ",Smith_average)
print("Steve Smith batting Strike rate against Dc bowlers : ",int((df_Smith['batsman_runs'].sum()/df_Smith.shape[0])*100))

Steve Smith average against DC bowlers :  24.0
Steve Smith batting Strike rate against Dc bowlers :  106


Actually his average looks decent i.e 24 but he scored only 48 runs. But his strike rate is not good i.e 106.

I am predicting he will score in the range of 16 - 30 in today's match.

### Q4) How many wide balls bowled in the match

In [80]:
#Here also i am taking individual players instead of team. Because team will change every 2years.
df_DCRR_bowlers = df_deliveries[((df_deliveries['bowler'] == 'R Ashwin') | (df_deliveries['bowler'] == 'K Rabada') |
                             (df_deliveries['bowler'] == 'AR Patel') | (df_deliveries['bowler'] == 'H Patel') | 
                             (df_deliveries['bowler'] == 'J Archer') | (df_deliveries['bowler'] == 'S Gopal') |
                             ((df_deliveries['bowler'] == 'R Tewatia')))] 
print("Number of wide balls bowled by present DC and RR bowlers : ",df_DCRR_bowlers['wide_runs'].sum()) 
print("Number of balls bowled by DC and RR bowlers : ", df_DCRR_bowlers.shape[0])

number_of_overs = df_DCRR_bowlers.shape[0]/6
print("number of overs bowled by DC and RR bowlers",number_of_overs)
widee_ball_per_over = int(number_of_overs/df_DCRR_bowlers['wide_runs'].sum())
print("For every",+widee_ball_per_over," overs they bowled wide ball")

Number of wide balls bowled by present DC and RR bowlers :  198
Number of balls bowled by DC and RR bowlers :  6603
number of overs bowled by DC and RR bowlers 1100.5
For every 5  overs they bowled wide ball


In [81]:
#So we have total 40 overs in the match
wide_balls_this_match = 40/widee_ball_per_over
print("wide balls bowled by DC and RR bowlers per match",wide_balls_this_match)

wide balls bowled by DC and RR bowlers per match 8.0


So Delhi and Rajasthan bowlers bowled 8 wide balls per match.

I am predicting in today match they bowl 6 or more wide balls.

### Q5) How many wickets will taken in total during the match

Actually number of wickets will be dependent on bowlers and Pitch as per my cricket knowledge. 
Ex :- CSK bowlers are very good at Chepauk(Chennai) pitch because it is spin friendly. I.e why they are struggled in 1st half of this season because UAE Pitches are bouncy and fast bowlers will get more purchase from those kind of pitches.

Today match is palying at Sharjah. Lets take average wickets fallen at sharjah cricket ground.

In [97]:
# For doing this task i am combining both Matches dataset and deliveries dataset 
#I am taking all the data from Matches dataset not only DC and RR data i.e why i am reading matches dataset one more time.
df_matches_1 = pd.read_csv("matches.csv")
#Merging both datasets
df_total = pd.merge(df_matches_1, df_deliveries, left_on='id',right_on='match_id', how='outer')

#Now lets calcualte how many wickets fallen at Sharjah cricket ground
df_sharjah = df_total[df_total['city'] == "Sharjah"]
df_sharjah_wickets = df_sharjah[df_sharjah['player_dismissed'] != 'No Wicket']

print("Total ",+ df_sharjah_wickets.shape[0], "number of wickets fallen in Sharjah cricket ground in", +df_sharjah['match_id'].nunique(),"matches")


Total  63 number of wickets fallen in Sharjah cricket ground in 6 matches


As per above observation per match in sharjah more than 10 wickets are fallen in single match.

So i am predicting in today match bowlers took 11-15 wickets.